# Notebook: Thu thập dữ liệu shop từ Pancake POS API

### 1.1. Import thư viện & Config

In [28]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
from sqlalchemy.types import BigInteger, DateTime
from sqlalchemy.dialects.mysql import LONGTEXT
# Load biến môi trường
load_dotenv()

API_KEY   = os.getenv("API_KEY")
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")


### 1.2. Kết nối tới database Bronze

In [23]:
# Tạo engine kết nối tới Bronze
engine_bronze = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}"
)

print(f"✅ Kết nối thành công tới database Bronze: {DB_BRONZE}")


✅ Kết nối thành công tới database Bronze: winner_bronze


### 1.3. Gọi API và lấy dữ liệu

In [24]:
url = f"https://pos.pages.fm/api/v1/shops?api_key={API_KEY}"
response = requests.get(url)

if response.status_code == 200:
    shops = response.json().get("shops", [])
    print(f"✅ Lấy được {len(shops)} shop")
else:
    print("❌ Lỗi:", response.status_code, response.text)
    shops = []


✅ Lấy được 1 shop


In [26]:
# Tạo DataFrame cho Bronze
df_bronze = pd.DataFrame([{
    "shop_id": shop["id"],
    "raw_json": json.dumps(shop, ensure_ascii=False),
    "extracted_at": datetime.now()
} for shop in shops])

display(df_bronze.head())


,shop_id,raw_json,extracted_at
0,230361475,"{""avatar_url"": ""https://content.pancake.vn/2-2...",2025-09-26 11:30:55.703722


In [30]:
df_bronze.to_sql(
    "shops_raw",
    con=engine_bronze,
    if_exists="append",   # append để lưu nhiều lần mà không mất dữ liệu cũ
    index=False,
    dtype={
        "shop_id":      BigInteger(),
        "raw_json":     LONGTEXT(),
        "extracted_at": DateTime()
    }
)

print(f"✅ Đã load {len(df_bronze)} dòng vào bảng `shops_raw` trong Bronze DB")


✅ Đã load 1 dòng vào bảng `shops_raw` trong Bronze DB
